In [ ]:
#@title ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title データを保存するディレクトリのパスを指定
#@markdown ランタイムに接続しドライブを後、コピーしてくる<br>
#@markdown 後ろの / は追加するので、追記は不要<br>

cwd = "/content/drive/MyDrive/RNAseq/Outputs/\u5B9F\u9A13\u540D" #@param {type:"string"}
cwd = cwd + "/"

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

In [ ]:
#@title 実験名を定義
exp_name = "\u5B9F\u9A13\u540D\u3092\u5165\u529B" #@param {type:"string"}
exp_name = exp_name + "_"

In [ ]:
#@title DEGと平均行を追加したTPMデータのパスを定義
df_tpm_path =  "\u5B9F\u9A13\u540D_DEGavetpm.csv\u30D5\u30A1\u30A4\u30EB\u306E\u30D1\u30B9\u3092\u5165\u529B" #@param {type:"string"}
df_tpm = pd.read_csv(df_tpm_path , index_col = 0)
df_tpm

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
読み込んだデータから、必要なデータを抽出する

In [ ]:
#@markdown 探すものの名前をカンマ区切りで入力<br>正規表現可、詳細は下のページを参考に<br>https://qiita.com/syuki-read/items/6c5f139abd676fcc0ca2
#@markdown 
drop_things = "NFE2L2, KEAP1" #@param {type:"string"}
name = drop_things
drop_things = drop_things.replace(' ', '')
drop_things = drop_things.split(',')

df_tyuusyutu = pd.DataFrame()

print("検索ワード:")
for i in range(len(drop_things)):
  print(drop_things[i], end="   ")
  df_tmp = df_tpm[df_tpm["product"].str.contains(drop_things[i] , regex=True)]
  df_tmp = df_tmp.sort_values("product")
  df_tyuusyutu = pd.concat([df_tyuusyutu, df_tmp])

df_tyuusyutu = df_tyuusyutu.drop_duplicates(subset='product')

# printして確認
print("\n=====================================")
print(df_tyuusyutu.to_string(index= False))

In [ ]:
# csvファイル形式で抽出してきた遺伝子を保存
df_tyuusyutu.to_csv(name + ".csv" , index = False)

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
グラフを描画する

In [ ]:
#@title 抽出した遺伝子のDEG_log2値をグラフ化
plt.figure(facecolor = "white" , figsize=(10, 5))
sns.barplot(x="DEG_log2" , y="product" , data = df_tyuusyutu)

In [ ]:
#@title N数の定義と必要データを纏めたdfの作成
n_num = 3 #@param {type:"integer"}
df = df_tyuusyutu.copy()

df

In [ ]:
#@title 平均値にエラーバー(標準誤差)をつけたグラフを横並びで描画
df_e = df.set_index("product")
df_e.drop(["DEG" , "DEG_log2"] , axis = 1 , inplace = True)
drop_index_list = []
for i in range(len(df_e)):
  if df_e.iloc[i , :].sum(axis = 0) == 0:
    drop_index_list = drop_index_list + [i]
df_e.drop(df_e.index[drop_index_list] , axis = 0 , inplace = True)    
# print(df_e)

index_name = list(df_e.index)
x = index_name
# print(index_name)

df_ave = df_e.iloc[: , [n_num , 2 * n_num + 1]]
df_e.drop(list(df_ave) , axis = 1 , inplace = True)

y_Cont = df_ave.iloc[: , 0]
y_Test = df_ave.iloc[: , 1]

df_eCont = df_e.iloc[: , 0:n_num]
# print(df_eCont)
df_eTest = df_e.iloc[: , n_num:2*n_num]
# print(df_eTest)

y_sdCont = df_eCont.std(1) / np.sqrt(n_num)
y_sdTest = df_eTest.std(1) / np.sqrt(n_num)

error_bar_options = {"lw":1 , "capthick":1 , "capsize": 5}
x_position = np.arange(len(index_name))
fig = plt.figure(facecolor = "white" , figsize=(len(x) * 1.2 , len(x) * 0.50))
ax = fig.add_subplot(1, 1, 1)
ax.bar(x_position      , y_Cont, width=0.4, yerr = y_sdCont , error_kw = error_bar_options , label='Cont')
ax.bar(x_position + 0.4, y_Test, width=0.4, yerr = y_sdTest , error_kw = error_bar_options ,  label='Test')
ax.legend()
ax.set_xticks(x_position + 0.2)
ax.set_xticklabels(index_name)
plt.show()

In [ ]:
#@title 各データの値をSeabornで比較
a = df_e.to_numpy().reshape(len(x) * 2 * n_num , 1)
b = list(df_e)

index_name = []
hue_name = []
for i in x:
  hue_name = hue_name + b
  for j in range(n_num * 2):
    index_name = index_name + [i]

fig = plt.figure(facecolor = "white" , figsize=(len(x) * 4 , len(x)))
ax = fig.add_subplot()
a = a.flatten()
sns.barplot(x = index_name , y = a , hue = hue_name)

In [ ]:
#@title Seabornでエラーバー描画
a = df_e.to_numpy().reshape(len(x) * 2 * n_num , 1)
b = []
for i in range(n_num):
  b = b + ["Cont"]
for i in range(n_num):
  b = b + ["Test"]

print(b)

index_name = []
hue_name = []
for i in x:
  hue_name = hue_name + b
  for j in range(n_num * 2):
    index_name = index_name + [i]

fig = plt.figure(facecolor = "white" , figsize=(len(x) * 1.5 , len(x) * 0.60))
ax = fig.add_subplot()
a = a.flatten()
sns.barplot(x = index_name , y = a , hue = hue_name , capsize=0.1 , errwidth=1.2)